# Counting DNA Nucleotides
-------------

## Problem

A string is simply an ordered collection of symbols selected from some alphabet and formed into a word; the length of a string is the number of symbols that it contains.

An example of a length 21 DNA string (whose alphabet contains the symbols 'A', 'C', 'G', and 'T') is "ATGCTTCAGAAAGGTCTTACG."

**Given**: A DNA string ss of length at most 1000 nt.

Return: Four integers (separated by spaces) counting the respective number of times that the symbols 'A', 'C', 'G', and 'T' occur in ss.

**Sample Dataset**:

    AGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAGAGTGTCTGATAGCAGC

**Sample Output**:

    20 12 17 21
    
-------------

## Solution

This seemingly simple problem actually provides some good insights on different data structures, performance differences based on built-in functions vs. loops, and big Oh notation. Let's start with the brute force approach: using the built-in 'count' function for each of the nucleotides.

In [1]:
# We will test on the sample data first
sample = "AGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAGAGTGTCTGATAGCAGC"

print(sample.count("A"), sample.count("C"), sample.count("G"), sample.count("T"))

20 12 17 21


The sample solution has a very specific format so calling 'count' four times is the quickest. However if instead of just counting the different nucleotides we were counting letter and digit frequency in a regular text it will be very tedious to make 30+ calls. If we don't care too much about formatting we can then use a loop:

In [2]:
# Create a string of all possible values
values = "ACGT"
counts = []
# Loop through and count
for nuc in values:
    counts.append((nuc, sample.count(nuc)))
print(counts)

[('A', 20), ('C', 12), ('G', 17), ('T', 21)]


How efficient is this brute force approach? In therms of space complexity, we don't need to make any copies or the original array, so it is _O_(1). In terms of time complexity, each time we count a specific letter we loop through the original string once, so in this case we will loop 4 times through the data. It is still an _O_(n) approach but in this case it is good to remember the constant. We will come back to it later when we actually time the performance.

Another approach that is perhaps more elegant is using a dictionary. A dictionary is a table that maps keys to values. In this case the keys of the dictionary are the different nucleotides and their values the number of times each of them appear on the input string.

In [3]:
# Create an empty dictionary
dic = {}
# Now as we loop through the input string we need to check if an entry in the dictionary already
# exists. If it does, we simply increase its value by one, otherwise we create it.
for nuc in sample:
    if nuc in dic.keys():
        dic[nuc] += 1
    else:
        dic[nuc] = 1
# Print the dictionary. NOTE: dictionaries are inherently unordered!
print(dic)

{'T': 21, 'A': 20, 'G': 17, 'C': 12}


Dictionaries are extremely useful, powerful, and elegant. So what is the complexity of this approach? This time we are creating a new data structure to store our frequencies. In this case we only have four different values, so the memory used is negligible but in general, if we had _d_ different values, we will need _O(d)_ space. Timewise, notice that using a dictionary we only have to loop through the input string once! So it only takes _O_(n) time. Can we make it more efficient? There is no way we can bring it down from _O_(n) but we can try to improve it by getting rid of the 'if', 'else' conditions inside the loop. There are two ways we can do it:

In [4]:
# Manually initialize the dictionary keys:

dic = {'A': 0, 'C': 0, 'T': 0, 'G': 0}
for nuc in sample:
    dic[nuc] += 1
print (dic)

{'G': 17, 'A': 20, 'C': 12, 'T': 21}


We can afford to do this because we only have four different keys. It is easy to see how this approach would be terribly inefficient and tedious when dealing with a larger number of keys. In that case we can use a defaultdict:

In [5]:
# Import defaultdict from the collections module
from collections import defaultdict

# We now initialize the defaultdict with an integer (whose default value is 0). What this does
# is that if you try getting the value of a previously unseen key, it will give you the 
# default value of 0, freeing us from the need to manually initiate each key.
dic = defaultdict(int)
for nuc in sample:
    dic[nuc] += 1
print (dic)

defaultdict(<class 'int'>, {'T': 21, 'A': 20, 'G': 17, 'C': 12})


Let's time this different approaches. We will use the timeit module, so first we have to wrap the approaches in functions:

In [6]:
def simple_print(s):
    a, c, g, t = s.count("A"), s.count("C"), s.count("G"), s.count("T")
    
def simple_loop(s):
    values = "ACGT"
    counts = []
    for nuc in values:
        counts.append((nuc, s.count(nuc)))
        
def normal_dic(s):
    dic = {}
    for nuc in s:
        if nuc in dic.keys():
            dic[nuc] += 1
        else:
            dic[nuc] = 1

def initialize_dic(s):
    dic = {'A': 0, 'C': 0, 'T': 0, 'G': 0}
    for nuc in s:
        dic[nuc] += 1
    
def default_dic(s):
    dic = defaultdict(int)
    for nuc in s:
        dic[nuc] += 1

In [7]:
# We can now use the timeit module to time them
import timeit
n = 100000

t = timeit.Timer(lambda: simple_print(sample))
print("Count statements:", t.timeit(number=n))

t = timeit.Timer(lambda: simple_loop(sample))
print("Count using a loop:", t.timeit(number=n))

t = timeit.Timer(lambda: normal_dic(sample))
print("Uninitialized dict:", t.timeit(number=n))

t = timeit.Timer(lambda: initialize_dic(sample))
print("Manualy initialized dict:", t.timeit(number=n))

t = timeit.Timer(lambda: default_dic(sample))
print("Default dict:", t.timeit(number=n))

Count statements: 0.189124755008379
Count using a loop: 0.22352015800424851
Uninitialized dict: 1.8125681559904478
Manualy initialized dict: 0.7410088409960736
Default dict: 0.9210935779847205


Some of the above results confirm our intuition: when using dictionaries, it is clear that either initializing all the keys, or using a defaultdic gives us better performance than checking whether a particular entry exists at each step. But how come that looping through the string 4 times, as in the first two approaches takes so little time? The built-in count function in python is written and precisely optimized in C, a lower-level and generally much faster language than python. So even though simple intuition tells us that looping once should be quicker than looping four times should be quicker, the difference in performance between a python loop and a C loop can be huge. But we may wonder if this depends on factors such as the lenght of the array and the number of items we are counting. What would happen if our input array was a million times larger?

In [8]:
sample_million = sample * 1000000
n = 1

t = timeit.Timer(lambda: simple_print(sample_million))
print("Count statements:", t.timeit(number=n))

t = timeit.Timer(lambda: simple_loop(sample_million))
print("Count using a loop:", t.timeit(number=n))

t = timeit.Timer(lambda: normal_dic(sample_million))
print("Uninitialized dict:", t.timeit(number=n))

t = timeit.Timer(lambda: initialize_dic(sample_million))
print("Manualy initialized dict:", t.timeit(number=n))

t = timeit.Timer(lambda: default_dic(sample_million))
print("Default dict:", t.timeit(number=n))

Count statements: 0.7458192369958851
Count using a loop: 0.7867667690152302
Uninitialized dict: 17.446918549016118
Manualy initialized dict: 7.996425088989781
Default dict: 7.910343224008102


The simple count statements are still faster, with ~10x better speed. Let's look at another practical application. A FASTQ file contains not only a nucleotide chain, but also a quality index associated for every entry in the chain. These quality indices can take the following values:

    !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz{|}~

So what would happen if instead of counting nucleotides we wanted to count how many times a particular quality index occurs? Instead of looking at four possible values ("ACTG") we are now looking at 94! Let's look at how the different approaches compare (we will just look at counting using a loop, uninitialized dict, and default dict as those are the only ones that make sense in this scenario):

In [9]:
# We need to redefine one of the functions:
def simple_loop_mod(s):
    values = """!"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz{|}~"""
    counts = []
    for nuc in values:
        counts.append((nuc, s.count(nuc)))

In [10]:
n = 1

t = timeit.Timer(lambda: simple_loop_mod(sample_million))
print("Count using a loop:", t.timeit(number=n))

t = timeit.Timer(lambda: normal_dic(sample_million))
print("Uninitialized dict:", t.timeit(number=n))

t = timeit.Timer(lambda: default_dic(sample_million))
print("Default dict:", t.timeit(number=n))

Count using a loop: 3.744830648007337
Uninitialized dict: 16.30485448401305
Default dict: 8.638961379008833


We now see that the performance of the dictionaries remains unchanged, but the performance of the simple count calls drops dramatically, going from a 10x improvement to a 2x.

We can now calculate the nucleotide frequency in the actual input using the simplest and fastest method. First we need to load the txt file:

In [11]:
# Open txt file, read into memory, and close file
f = open('rosalind_dna.txt', 'r')
s = f.readline()

print(s.count("A"), s.count("C"), s.count("G"), s.count("T"))

199 221 204 205


_______________
In this short notebook we have seen the difference in performance between the built-in count method and more advanced but very common techniques such as dictionaries. The "C" backbone and precise optimization of the built-in python function make the simpler technique faster in than dictionaries in most cases. Caution is advised though, since this may not be the case for very large input strings and very large numbers of items to count (which translates into loops through the input).